In [5]:
import os, json, cv2, numpy as np, matplotlib.pyplot as plt, pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F

import albumentations as A # Library for augmentations

In [61]:
# setting seeds for reproducability
torch.manual_seed(69)

import random
random.seed(69)

np.random.seed(69)

# change the number of keypoints in cocotools

C:\Users\CallumMacpherson\anaconda3\envs\park\Lib\site-packages\pycocotools

# 1. Imports

In [6]:
# https://github.com/pytorch/vision/tree/main/references/detection
import transforms, utils, engine, train
from utils import collate_fn
from engine import train_one_epoch, evaluate

# 2. Setting up the dataset

In [7]:
anno_dir = "../BBox_keypoint.csv"

In [10]:
labels_df = pd.read_csv(anno_dir, index_col=0)

In [12]:
labels_df.head()

,patient ID,img_no,finger x,finger y,thumb x,thumb y,wristlow x,wristlow y,wristhigh x,wristhigh y,knuckle1 x,knuckle1 y,knuckle2 x,knuckle2 y,class,x1,y1,x2,y2
0,OC01_R_10s_lowfps,img000.png,527.435088,643.753157,568.125216,694.943964,826.704422,924.646304,909.397265,822.264690,795.202387,652.941250,648.192889,637.190233,0,121.663802,113.942607,1241.663802,1233.942607
1,OC01_R_10s_lowfps,img001.png,535.072768,643.354545,567.283288,691.011391,832.045460,928.036573,910.213339,824.653249,801.786926,645.623590,650.494256,638.058957,0,124.535107,108.317490,1244.535107,1228.317490
2,OC01_R_10s_lowfps,img002.png,533.811996,637.657582,567.946800,693.272491,830.170829,923.506428,912.121025,820.123104,797.390751,648.658079,653.662715,638.571901,0,122.668775,110.965285,1242.668775,1230.965285
3,OC01_R_10s_lowfps,img003.png,527.324229,636.657715,567.484718,691.878388,824.762853,921.546186,913.868939,821.144963,793.387471,651.717898,649.060712,639.167745,0,120.436095,111.798143,1240.436095,1231.798143
4,OC01_R_10s_lowfps,img004.png,528.169044,636.975038,567.431449,691.435793,808.071996,911.811873,908.127803,814.289125,797.939763,647.107271,652.288905,639.508096,0,122.685606,109.271532,1242.685606,1229.271532


## We want to rename all the files to patient ID + img_no

In [20]:
data_dir = "../data/labelledall vids"

In [58]:
#renamng files
for root, dirs, files in os.walk(data_dir):
    # print(dirs)
    for d in (dirs):
        # print(os.path.join(root,d))
        for r2, d2, f2 in os.walk(os.path.join(root, d)):
            # print(r2, d2, f2)
            for file in f2:
                # print(file)
                
                name = (d + "_" + file)
                os.rename(os.path.join(r2, file), os.path.join(r2,name))
    #         break
    #     break
    # break

In [62]:
outdir = "../data/fullname"

for dirpath, dirnames, filenames in os.walk(data_dir):
    # print(dirpath, dirnames, filenames)
    for filename in [f for f in filenames if f.endswith(".png")]:
        old_temp = os.path.join(dirpath, filename)
        new_temp = os.path.join(outdir, filename)
        os.rename(old_temp, new_temp)
        # print(new_temp)

# Making Train and Test dirs

#90% train, 10% test